<a href="https://colab.research.google.com/github/Sharique121/Data-Visualisation-Practice/blob/master/Market_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
from google.colab import files
datafile = files.upload()

Saving Sales_data.csv to Sales_data.csv


In [0]:
import io

In [0]:
data=pd.read_csv(io.StringIO(datafile['Sales_data.csv'].decode('utf-8')))

In [6]:
data.head()

,Month,Billing No.,GST Invoice No.,Customer,Customer Name,Dis.Channel,Division,Mat.Pricing Grp.Desc,Cust Grp.,Billing Date,Quantity,Sum of NSr,Sum of Nep,State
0,April.19,9000012594,F21912200087,500000,BHUSHAN POWER & STEEL LIMITED,10,20,HS ROD,LIST PRICE,06-Apr-19,31,4471293.0,7694,Odisha
1,April.19,9000012692,F21912200111,500000,BHUSHAN POWER & STEEL LIMITED,10,20,HS ROD,LIST PRICE,08-Apr-19,20,2986606.0,5139,Odisha
2,April.19,9000012693,F21912200112,500000,BHUSHAN POWER & STEEL LIMITED,10,20,HS ROD,LIST PRICE,08-Apr-19,11,1602971.0,2758,Odisha
3,April.19,9000012817,F21912200147,500000,BHUSHAN POWER & STEEL LIMITED,10,20,HS ROD,LIST PRICE,10-Apr-19,19,2715574.0,4673,Odisha
4,April.19,9000012911,F21912200196,500000,BHUSHAN POWER & STEEL LIMITED,10,20,HS ROD,LIST PRICE,11-Apr-19,20,2981641.0,5130,Odisha


In [7]:
data.columns

Index(['Month', 'Billing No.', 'GST Invoice No.', 'Customer', 'Customer Name',
       'Dis.Channel', 'Division', 'Mat.Pricing Grp.Desc', 'Cust Grp.',
       'Billing Date', 'Quantity', 'Sum of NSr', 'Sum of Nep', 'State'],
      dtype='object')

In [8]:
data_new=data[['Customer', 'Billing Date', 'Sum of NSr']]
data_new.head()

,Customer,Billing Date,Sum of NSr
0,500000,06-Apr-19,4471293.0
1,500000,08-Apr-19,2986606.0
2,500000,08-Apr-19,1602971.0
3,500000,10-Apr-19,2715574.0
4,500000,11-Apr-19,2981641.0


In [9]:
data=data_new
data.head()

,Customer,Billing Date,Sum of NSr
0,500000,06-Apr-19,4471293.0
1,500000,08-Apr-19,2986606.0
2,500000,08-Apr-19,1602971.0
3,500000,10-Apr-19,2715574.0
4,500000,11-Apr-19,2981641.0


In [10]:
data.columns=['Customer_ID', 'Date', 'Bill_Value']
data.head()

,Customer_ID,Date,Bill_Value
0,500000,06-Apr-19,4471293.0
1,500000,08-Apr-19,2986606.0
2,500000,08-Apr-19,1602971.0
3,500000,10-Apr-19,2715574.0
4,500000,11-Apr-19,2981641.0


In [0]:
from datetime import datetime
#d = datetime.strptime(data["Date"], "%Y-%m-%d %H:%M:%S")

In [0]:
data["Date"]=pd.to_datetime(data["Date"])

In [13]:
data.head()

,Customer_ID,Date,Bill_Value
0,500000,2019-04-06,4471293.0
1,500000,2019-04-08,2986606.0
2,500000,2019-04-08,1602971.0
3,500000,2019-04-10,2715574.0
4,500000,2019-04-11,2981641.0


In [0]:
data["Bill_Value"]=pd.to_numeric(data["Bill_Value"])
data["Customer_ID"]=pd.to_numeric(data["Customer_ID"])

In [15]:
data.head()
data["Customer_ID"].count()

12253

In [16]:
data.describe()

,Customer_ID,Bill_Value
count,12253.000000,1.225300e+04
mean,502091.587366,2.630729e+06
std,1080.383678,1.405318e+06
min,500000.000000,2.739824e+04
25%,501426.000000,1.450624e+06
50%,502326.000000,2.843080e+06
75%,502641.000000,3.748251e+06
max,504485.000000,6.522598e+06


# **Recency**


In [17]:
#group by customers and check last date of purshace
recency_df = data.groupby(by='Customer_ID', as_index=False)['Date'].max()
recency_df.columns = ['Customer_ID','LastPurshaceDate']
recency_df.head()

,Customer_ID,LastPurshaceDate
0,500000,2019-12-31
1,500016,2019-12-18
2,500024,2019-12-28
3,500030,2019-12-31
4,500036,2019-12-31


# Calculating Recency

In [18]:
from datetime import datetime as dt
now=(data["Date"].max())
now

Timestamp('2019-12-31 00:00:00')

In [0]:
recency_df['Recency'] = recency_df['LastPurshaceDate'].apply(lambda x: (now - x).days)

In [20]:
recency_df.head() #this is the no of days since the last purchase

,Customer_ID,LastPurshaceDate,Recency
0,500000,2019-12-31,0
1,500016,2019-12-18,13
2,500024,2019-12-28,3
3,500030,2019-12-31,0
4,500036,2019-12-31,0


In [0]:
#now let us delete LastPurchaseDate as we do not need it any more we will be dealing with only the recency value
recency_df.drop(["LastPurshaceDate"], axis=1, inplace=True)

In [22]:
recency_df.head()

,Customer_ID,Recency
0,500000,0
1,500016,13
2,500024,3
3,500030,0
4,500036,0


# **Frequency**

In [23]:
frequency=data.groupby(by=['Customer_ID'], as_index=False)['Bill_Value'].count()
frequency.columns=["Customer_ID", "Frequency"]
frequency.head()

,Customer_ID,Frequency
0,500000,209
1,500016,4
2,500024,17
3,500030,20
4,500036,12


# **Monetary**
How much money did the customer spent over all

In [24]:
avg=data.groupby("Customer_ID", as_index=False)['Bill_Value'].mean()
avg.columns=["Customer_ID", "Monetary"]
avg.head()

,Customer_ID,Monetary
0,500000,3.613412e+06
1,500016,2.890396e+06
2,500024,3.531051e+06
3,500030,3.384489e+06
4,500036,3.312511e+06


# **Creating RFM Table**

In [26]:
rfm=recency_df.merge(frequency, on="Customer_ID")
rfm.head() 

,Customer_ID,Recency,Frequency
0,500000,0,209
1,500016,13,4
2,500024,3,17
3,500030,0,20
4,500036,0,12


In [27]:
rfm2=rfm.merge(avg, on="Customer_ID")
rfm2.head()

,Customer_ID,Recency,Frequency,Monetary
0,500000,0,209,3.613412e+06
1,500016,13,4,2.890396e+06
2,500024,3,17,3.531051e+06
3,500030,0,20,3.384489e+06
4,500036,0,12,3.312511e+06


# **EDA**

In [28]:
#no of transactions through the year
b=data.groupby(["Date"], as_index=False)["Customer_ID"].count()
b.head() # Day wise group

,Date,Customer_ID
0,2019-04-02,3
1,2019-04-03,8
2,2019-04-04,13
3,2019-04-05,13
4,2019-04-06,27


In [29]:
rfm2.describe()

,Customer_ID,Recency,Frequency,Monetary
count,318.000000,318.000000,318.000000,3.180000e+02
mean,502676.194969,61.588050,38.531447,2.145886e+06
std,1122.798113,71.226414,72.444370,1.198336e+06
min,500000.000000,0.000000,1.000000,1.413853e+05
25%,502212.500000,4.000000,3.000000,1.140873e+06
50%,502589.000000,33.000000,12.000000,2.145445e+06
75%,503477.500000,96.000000,37.000000,3.160116e+06
max,504485.000000,273.000000,452.000000,5.204094e+06


# **Customer segments with RFM Model**

The simplest way to create customers segments from RFM Model is to use Quartiles. We assign a score from 1 to 4 to Recency, Frequency and Monetary. Four is the best/highest value, and one is the lowest/worst value. A final RFM score is calculated simply by combining individual RFM score numbers.

Note: Quintiles (score from 1-5) offer better granularity, in case the business needs that but it will be more challenging to create segments since we will have 555 possible combinations. So, we will use quartiles.

In [30]:
rfm_df =rfm2
rfm_df.set_index('Customer_ID',inplace=True)
rfm_df.head()

,Recency,Frequency,Monetary
Customer_ID,,,
500000,0,209,3.613412e+06
500016,13,4,2.890396e+06
500024,3,17,3.531051e+06
500030,0,20,3.384489e+06
500036,0,12,3.312511e+06


In [31]:
quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
quantiles

,Recency,Frequency,Monetary
0.25,4.0,3.0,1.140873e+06
0.50,33.0,12.0,2.145445e+06
0.75,96.0,37.0,3.160116e+06


In [32]:
quantiles.to_dict()


{'Frequency': {0.25: 3.0, 0.5: 12.0, 0.75: 37.0},
 'Monetary': {0.25: 1140872.9411989474,
  0.5: 2145444.69515,
  0.75: 3160115.5187311945},
 'Recency': {0.25: 4.0, 0.5: 33.0, 0.75: 96.0}}

# **Creation of RFM Segments**

We will create two segmentation classes since, high recency is bad, while high frequency and monetary value is good.

In [0]:
# Arguments (x = value, p = recency, monetary_value, frequency, d = quartiles dict)
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [0]:
#create rfm segmentation table
rfm_segmentation = rfm_df
rfm_segmentation['R_Quartile'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantiles,))


In [0]:
rfm_segmentation['M_Quartile'] = rfm_segmentation['Monetary'].apply(FMScore, args=('Monetary',quantiles,))

In [36]:
rfm_segmentation.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile
Customer_ID,,,,,,
500000,0,209,3.613412e+06,4,4,4
500016,13,4,2.890396e+06,3,2,3
500024,3,17,3.531051e+06,4,3,4
500030,0,20,3.384489e+06,4,3,4
500036,0,12,3.312511e+06,4,2,4


Now that we have the score of each customer, we can represent our customer segmentation. First, we need to combine the scores (R_Quartile, F_Quartile,M_Quartile) together.

In [37]:
rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \
                            + rfm_segmentation.F_Quartile.map(str) \
                            + rfm_segmentation.M_Quartile.map(str)
rfm_segmentation.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
Customer_ID,,,,,,,
500000,0,209,3.613412e+06,4,4,4,444
500016,13,4,2.890396e+06,3,2,3,323
500024,3,17,3.531051e+06,4,3,4,434
500030,0,20,3.384489e+06,4,3,4,434
500036,0,12,3.312511e+06,4,2,4,424


Best Recency score = 4: most recently purchase. Best Frequency score = 4: most quantity purchase. Best Monetary score = 4: spent the most.

Let's see who are our Champions (best customers).

In [38]:
rfm_segmentation[rfm_segmentation['RFMScore']=='444'].sort_values('Monetary', ascending=False).head(10)

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
Customer_ID,,,,,,,
503357,0,93,4.802826e+06,4,4,4,444
502081,0,161,4.601512e+06,4,4,4,444
502082,0,167,4.472288e+06,4,4,4,444
503678,1,41,3.798615e+06,4,4,4,444
500482,0,293,3.712543e+06,4,4,4,444
501226,0,97,3.677692e+06,4,4,4,444
500000,0,209,3.613412e+06,4,4,4,444
503112,0,159,3.563161e+06,4,4,4,444
500096,0,391,3.473580e+06,4,4,4,444


# **How many customers do we have in each segment?**

In [39]:
print("Best Customers: ",len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']))
print('Loyal Customers: ',len(rfm_segmentation[rfm_segmentation['F_Quartile']==4]))
print("Big Spenders: ",len(rfm_segmentation[rfm_segmentation['M_Quartile']==4]))
print('Almost Lost: ', len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']))
print('Lost Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']))
print('Lost Cheap Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='111']))

Best Customers:  14
Loyal Customers:  79
Big Spenders:  80
Almost Lost:  7
Lost Customers:  4
Lost Cheap Customers:  18


Now that we knew our customers segments we can choose how to target or deal with each segment.

For example:

Best Customers - Champions: Reward them. They can be early adopters to new products. 

At Risk: Send them personalized emails to encourage them to shop.